In [17]:
from pyhwpx import Hwp

In [18]:
hwp = Hwp()

In [22]:
# 스타일 찾기 함수 지정
def search_style(style_number):
	hwp.HAction.GetDefault("Goto", hwp.HParameterSet.HGotoE.HSet);
	hwp.HParameterSet.HGotoE.HSet.SetItem("DialogResult", style_number);
	hwp.HParameterSet.HGotoE.SetSelectionIndex = 4;
	hwp.HAction.Execute("Goto", hwp.HParameterSet.HGotoE.HSet);

In [47]:
# 블록 텍스트 추출
def extract_text_from_block():  # 이 함수명을 영어로 바꾸면 = extract_text_from_block
    hwp.InitScan(range=0xff)  # 0xff <<선택된 범위 내에서 검색
    _, text_blokced = hwp.GetText()  # 텍스트만 추출
    hwp.ReleaseScan() # 스캔을 해제.
    return text_blokced # 이경우, 해당 target이 text에 포함되어있으면 True, 아니면 False를 반환.

In [277]:
def search_text(text,direction="Forward"):
    """
    특정 텍스트를 찾아서 블록 처리
    Args:
        text (str): 찾을 텍스트
        direction (optional): 찾기 방향(Forward, Backward, AllDoc)
    """
    
    hwp.HAction.GetDefault("RepeatFind", hwp.HParameterSet.HFindReplace.HSet)
    hwp.HParameterSet.HFindReplace.FindString = f"{text}"
    hwp.HParameterSet.HFindReplace.Direction = hwp.FindDir(f"{direction}")
    hwp.HParameterSet.HFindReplace.FindType = 1  
    hwp.HParameterSet.HFindReplace.UseWildCards = 1
    hwp.HParameterSet.HFindReplace.IgnoreMessage = 2  # 계속 찾을까요? 1=YES, 2=NO
    hwp.HAction.Execute("RepeatFind", hwp.HParameterSet.HFindReplace.HSet)
    
    
    
def search_text_condition(text): 
    """조건식사용에 체크하고 텍스트 검색

	Args:
		text (_type_): _description_
	"""
    hwp.SetMessageBoxMode(0x00020000) # 계속 찾으시겠습니까? 아니오
    hwp.HAction.GetDefault("RepeatFind", hwp.HParameterSet.HFindReplace.HSet)
    hwp.HParameterSet.HFindReplace.FindString = f"{text}"
    hwp.HParameterSet.HFindReplace.Direction = hwp.FindDir("Forward")
    hwp.HParameterSet.HFindReplace.FindType = 1
    hwp.HParameterSet.HFindReplace.FindRegExp = 1 
    hwp.HParameterSet.HFindReplace.IgnoreMessage = 2
    if hwp.HAction.Execute("RepeatFind", hwp.HParameterSet.HFindReplace.HSet):
        hwp.SetMessageBoxMode(0x000F0000) # 기본값으로 돌리기기
        return True
    else:
        hwp.SetMessageBoxMode(0x000F0000) # 기본값으로 돌리기기
        return False


In [20]:
###### 문항 채점 과정
# (1) 객관식 문단번호 스타일을 찾아가 원문 출처, 총 문항 수, 배점, 정답 번호를 구하고 문항 번호를 부여한다.
# (2) 주관식 문단번호 스타일을 찾아 발문, 배점을 구하고 발문 좌표 저장
# (3) 발문에서 GetHEadString()으로 논술형 번호 추출 -> 문서 끝으로 이동한 뒤 논술형 번호로 "위로 찾기"
# (4) 논술형 정답을 추출하고 다시 발문 좌표로 이동

In [149]:
def extract_number_from_text(text):
    """
    텍스트에서 숫자(정수 또는 소수)를 추출하는 함수
    
    Args:
        text (str): 숫자가 포함된 텍스트
        
    Returns:
        float: 추출된 숫자. 숫자가 없으면 None 반환
    """
    import re
    
    number_match = re.search(r'(\d+\.?\d*)', text)
    if number_match:
        return float(number_match.group(1))
    return None

In [190]:
def convert_circled_number(text):
    """
    다양한 원문자를 일반 숫자로 변환하는 함수
    
    Args:
        text (str): 원문자가 포함된 텍스트
        
    Returns:
        int: 변환된 숫자
    """
    # 다양한 원문자 매핑
    circled_num_map = {
        '①': 1, '❶': 1, '➀': 1,  # 1에 해당하는 다양한 원문자
        '②': 2, '❷': 2, '➁': 2,  # 2에 해당하는 다양한 원문자
        '③': 3, '❸': 3, '➂': 3,  # 3에 해당하는 다양한 원문자
        '④': 4, '❹': 4, '➃': 4,  # 4에 해당하는 다양한 원문자
        '⑤': 5, '❺': 5, '➄': 5,  # 5에 해당하는 다양한 원문자
        # 필요한 만큼 추가
    }
    
    # 입력된 텍스트에서 원문자를 찾아 변환
    for circle_num, num in circled_num_map.items():
        if circle_num in text:
            return num
    
    return None

In [182]:
# 미주 진입
def enter_note():
    hwp.Run("MoveParaBegin")
    hwp.SelectCtrlFront()
    hwp.Run("MoveRight")
    hwp.FindCtrl()
    hwp.HAction.Run("NoteModify")


In [285]:
#if __name__ == "__main__":

# 한글 실행
from pyhwpx import Hwp
hwp = Hwp()
hwp.Run("MoveDocBegin")

True

In [210]:
유형_식별자_리스트 = ["객관식", "주관식", "논술형"]

In [286]:
# 객관식 문단번호 찾아서 블록 처리
search_style(2)
hwp.Run("MoveSelParaEnd")

True

In [287]:
# 발문 저장
block_text = extract_text_from_block()
print(block_text)
print("발문추출완료")

다음 글의 내용과 일치하지 않는 것은?


In [288]:
enter_note()

In [289]:
#정답 추출
search_text("*")
answer_circled = extract_text_from_block()
answer_num = convert_circled_number(answer_circled)
print(answer_num)

3


In [290]:
# 해설 블록 설정
hwp.Run("MoveParaEnd")
for i in range(30):
    hwp.Run("MoveSelNextParaBegin")
# 해설 추출
explanation_text = extract_text_from_block()
print(f"해설 추출 완료")    

In [291]:
# 미주 종료
hwp.Run("CloseEx")
#점수 추출


True

In [292]:
#발문 행 블럭
hwp.Run("MoveParaBegin")
hwp.Run("MoveSelParaEnd")

True

In [293]:
# 발문 행 블럭 -> 배점 추출
search_text_condition(r"{\d점}|{\d\.\d점}")
score_text = extract_text_from_block()
score = extract_number_from_text(score_text)
hwp.Run("Cancel")
print("배점추출완료")

3.8


In [298]:
from pyhwpx import Hwp
hwp = Hwp()


In [301]:
for i in range(30):
        hwp.Run("MoveSelNextParaBegin")    